In [55]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
import json
import csv

options = Options()
options.add_argument('--headless')  # Run in headless mode (no GUI)
options.add_argument('--disable-gpu')  # Disable GPU acceleration
options.add_argument('--no-sandbox')  # Bypass OS security model
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
options.add_argument(f"user-agent={user_agent}")
download_dir = "./loj/testdata" 
prefs = {
	"download.default_directory": download_dir,
	"download.prompt_for_download": False, 
	"download.directory_upgrade": True, 
	"safebrowsing.enabled": True 
}
options.add_experimental_option("prefs", prefs)

# Path to ChromeDriver
chromedriver_path = "/home/jianzhu/LLm-pddl-benchmark/code/chromedriver-linux64/chromedriver"  # Replace with your actual path
service = Service(chromedriver_path)

driver = webdriver.Chrome(service=service, options=options)

driver.set_page_load_timeout(12000) 

url = "https://loj.ac/p"
driver.get(url)
time.sleep(5)  # Allow time for the page to load

print(f"Crawling: {url}")

Crawling: https://loj.ac/p


In [ ]:
pagination = driver.find_elements(By.CSS_SELECTOR, "div._pagination_1ip09_1.menu a.item")
total_pages = int(pagination[-2].text)  # 倒数第二个是总页数
print(f"Total pages: {total_pages}")


In [ ]:
import json
base_url = "https://loj.ac/p"
all_problems = []
for page in range(1, total_pages + 1):
	print(f"Scraping page {page}...")
	page_url = f"{base_url}?page={page}"
	driver.get(page_url)

	time.sleep(2)

	rows = driver.find_elements(By.CSS_SELECTOR, "tr._row_15kiv_90")
	for row in rows:
		columns = row.find_elements(By.TAG_NAME, "td")
		if len(columns) >= 4:
			problem_id = columns[0].text
			title_element = columns[1].find_element(By.TAG_NAME, "a")
			title = title_element.text
			link = "https://loj.ac" + title_element.get_attribute("href")
			submissions = columns[2].text
			acceptance = columns[3].text

			problem_data = {
				"ID": problem_id,
				"Title": title,
				"Link": link,
				"Submissions": submissions,
				"Acceptance": acceptance,
			}
			all_problems.append(problem_data)
output_file = "problems.json"
with open(output_file, "w", encoding="utf-8") as f:
	json.dump(all_problems, f, ensure_ascii=False, indent=4)
print(f"Data has been saved to {output_file}")

In [ ]:
import json
import os
from tqdm import tqdm
problems = json.load(open("problems.json","r"))
for problem in tqdm(problems):

	url = problem["Link"]
	# print(url)
	driver.get(url)
	time.sleep(2)
	if int(problem["ID"]) < 100:
		continue
	title_element = driver.find_element(By.CSS_SELECTOR, "h1.ui.header._header_1rcs8_97 > span")
	title = title_element.text.strip()
	
	# skip
	file_name = f"loj/{title.replace(' ', '_').replace("/", "_")}.json"
	if os.path.exists(file_name):
		print(f"Skip {title}")
		continue

	labels_section = driver.find_element(By.CSS_SELECTOR, "div._labels_1rcs8_107")

	try:
		show_tags_button = labels_section.find_element(By.CSS_SELECTOR, "a._toggleTagsLabel_1rcs8_113")
		show_tags_button.click()
		time.sleep(1)
	except Exception:
		pass

	labels = []
	labels_elements = labels_section.find_elements(By.CSS_SELECTOR, "div.ui.label, a.ui.label")
	for label in labels_elements:
		labels.append(label.text.strip())

	# 获取样例对（输入和输出）
	samples = []
	sample_elements = driver.find_elements(By.CLASS_NAME, "_sampleDataPre_1rcs8_198")
	current_key = "input"
	temp_dict = {}
	for sample_element in sample_elements:
		text = sample_element.text
		temp_dict[current_key] = text
		if current_key == "output":
			samples.append(temp_dict)
			temp_dict = {}
		current_key = "output" if current_key == "input" else "input"
	

	# 遍历 `_statementView_1rcs8_158` 容器中的所有子元素并提取文本
	statement_container = driver.find_element(By.CSS_SELECTOR, "div._statementView_1rcs8_158")
	statement_elements = statement_container.find_elements(By.XPATH, "./*")

	statement_data = {}
	for element in statement_elements:
		tag_name = element.tag_name
		text_content = element.text.strip()
		if text_content:  # 只记录非空文本
			statement_data[tag_name] = statement_data.get(tag_name, [])  # 初始化键
			statement_data[tag_name].append(text_content)


	content_sections = driver.find_elements(By.CSS_SELECTOR, "div._markdownContent_1lggv_1")
	content = {}
	headers = driver.find_elements(By.CSS_SELECTOR, "div.ui.large.header")
	for i, section in enumerate(content_sections):
		header_text = headers[i].text.strip() if i < len(headers) else f"Section {i+1}"
		content[header_text] = section.text.strip()

	problem_data = {
		"ID": problem["ID"],
		"title": title,
		"labels": labels,
		"samples": samples,
		"content": content,
		"submissions": problem["Submissions"],
		"acceptance": problem["Acceptance"],
	}

	driver.get(f"{url}/files")
	time.sleep(2)

	# Download test cases
	check_all_checkbox = driver.find_element(By.XPATH, "/html/body/div[1]/div[4]/div/div[1]/div/table/thead/tr/th[1]/div")
	if not check_all_checkbox.is_selected():
		check_all_checkbox.click()
		time.sleep(1)
	time.sleep(1)
	download_button = driver.find_element(By.XPATH, "/html/body/div[1]/div[4]/div/div[1]/div/table/tfoot/tr/th/div/div/div")
	download_button.click()
	time.sleep(1)
	download_button = driver.find_element(By.CSS_SELECTOR, "div.menu.transition.visible._fileTableSelectedFilesDropdownMenu_r2613_51 div.item")
	driver.execute_script("arguments[0].click();", download_button)

	file_name = f"TestData_#{problem["ID"]}.zip"
	while True:
		if file_name in os.listdir(download_dir):
			break
		time.sleep(1)

	output_file = f"loj/{title.replace(' ', '_').replace("/","_")}.json"
	with open(output_file, "w", encoding="utf-8") as f:
		json.dump(problem_data, f, ensure_ascii=False, indent=4)

	import os

	current_directory = os.getcwd()
	total_size = 0

	for filename in os.listdir(current_directory):
		file_path = os.path.join(current_directory, filename)
		
		if os.path.isfile(file_path):
			file_size = os.path.getsize(file_path)
			total_size += file_size

	if total_size / 1024**3 > 500:
		# exit
		break

	print(f"Data has been saved to {output_file}")

  0%|          | 9/3353 [00:29<4:12:24,  4.53s/it]

Data has been saved to loj/#100._矩阵乘法.json


  0%|          | 10/3353 [00:38<5:30:26,  5.93s/it]

Data has been saved to loj/#101._最大流.json


  0%|          | 11/3353 [00:47<6:22:54,  6.87s/it]

...

Data has been saved to loj/#2051._「HNOI2016」序列.json


  7%|▋         | 220/3353 [45:31<36:14:31, 41.64s/it]

Data has been saved to loj/#2052._「HNOI2016」矿区.json


  7%|▋         | 221/3353 [46:21<38:35:53, 44.37s/it]

Data has been saved to loj/#2053._「HNOI2016」大数.json


In [ ]:
import os

current_directory = os.getcwd()
total_size = 0

for filename in os.listdir(current_directory):
	file_path = os.path.join(current_directory, filename)
	
	if os.path.isfile(file_path):
		file_size = os.path.getsize(file_path)
		total_size += file_size

print(f"Total size of all files: {total_size / 1024:.2f} KB, {total_size / 1024**2:.2f} MB, {total_size / 1024**3:.2f} GB")

In [ ]:
driver.get("https://loj.ac/p/110")
time.sleep(4)

In [ ]:
data_element = driver.find_elements(By.CLASS_NAME, "_sampleDataPre_1rcs8_198")
for i in data_element:
    extracted_text = i.text  # 提取文本内容
    print(extracted_text)

In [ ]:
driver.quit()